In [1]:
import pandas as pd
import numpy as np
import random as rd
from sklearn.cluster import AgglomerativeClustering 
from sklearn.metrics.cluster import v_measure_score
from anytree import Node, RenderTree
from sklearn.preprocessing import normalize
from random import randrange

In [ ]:
#operacao div para evitar divisao por 0
def div(a,b):
    if(b==0):
        return 1
    else:
        return a/b

In [ ]:
#dericavao da gramatica
def derivacao(nterm, num,dp = 0):#add nvars
    if(nterm[0] != '<'):
        return 't'
    
    if(nterm == '<op>'):
        if(num%3 == 0):
            return ( '+')
        elif(num%3 == 1):
            return ('-')
        elif(num%3 == 2):
            return ('*')
        
    if(nterm == '<protop>'):
        return ('div')
            
    if(dp == 1):
        return (f'x{num%8}')
    
    if(nterm == '<start>'):
        if(num%3 == 0):
            return ( '<exp> <op> <exp>')
        if(num%3 == 1):
            return ('<exp>')
        if(num%3 == 2):
            return ( '( <exp> <op> <exp> )')
        
    if(nterm == '<exp>'):
        if(num%4 == 0):
            return ('<exp> <op> <exp>')
        elif(num%4 == 1):
            return ('( <exp> <op> <exp> )')
        elif(num%4 == 2):
            return ( '<protop> ( <term> , <term> )')
        elif(num%4 == 3):
            return ('<term>')
            

    if(nterm == '<term>'):
        return (f'x{num%8}')
        


In [ ]:
#transforma o individuo em uma lista de numeros para facilitar acesso as particoes
def dividir_binario(numero, tam_seg=8):
    binario = bin(numero)[2:]
    binario = binario.zfill(((len(binario) + tam_seg - 1) // tam_seg) * tam_seg)
    
    segmentos = [binario[i:i + tam_seg] for i in range(0, len(binario), tam_seg)]
    return [int(segmento, 2) for segmento in segmentos]

#implementacao de uma variavel global para iterar pelo binario do individuo
i = 0

def incrementar():
    global i
    i += 1   
    
def reset():
    global i
    i = 0

def check():
    return i

#operacoes bitwise

def bit_set(n,pos):
    return n | (1 << pos)

def bit_clear(n, pos):
    return n & ~(1 << pos)

def bit_switch(n, pos):
    return n ^ (1 << pos)
    

In [ ]:
#criacao de arvore de derivacao da gramatica e suas operacoes
def ConstructTree(nums,no = '<start>', pai=None, dp = 0):
    if(check() >= len(nums)):
        reset()
    no = Node(no, parent=pai)
    if(dp == 6):
        next = derivacao(no.name, nums[i], 1)
    else:
        next = derivacao(no.name, nums[i])
    incrementar()
    if(next == 't'):
        return no
    for s in next.split():
        ConstructTree(nums,s,no,dp+1)
        
    return no
       

def ImprimeArvore(raiz):
    for pre, _, node in RenderTree(raiz):
        print(f"{pre}{node.name}")

def armazena_folhas(raiz):
    folhas = []
    for pre, _, n in RenderTree(raiz):
        if not n.children:
            folhas.append(n.name)  
    return folhas

In [ ]:
#interpreta as expressoes dos valores usando a arvore
def criar_expressoes(pop):
    expressoes = []
    for n in pop:
        reset()
        arvore = ConstructTree(dividir_binario(n))
        folhas = armazena_folhas(arvore)
        expressoes.append(' '.join(folhas))
    return expressoes


#cria uma pop de individuos de até 64 bits
def Pop_start(size):
    pop = []
    for i in range(size):
        pop.append(randrange(0,18446744073709551616)) # pow(2, 64) = 18446744073709551616
    return pop


In [7]:
#calcula uma expressao usando as variaveis x1 a x8 
# sendo xi = val1[ponto1] - val2[pont2]
def calc (expr, x):
    x0 = x.iloc[0]
    x1 = x.iloc[1]
    x2 = x.iloc[2]
    x3 = x.iloc[3]
    x4 = x.iloc[4]
    x5 = x.iloc[5]
    x6 = x.iloc[6]
    x7 = x.iloc[7]
 
    return eval(expr)


#calcula matriz de distancia pra um individuo
def calc_Matriz(expr,df):
  Mdist = []
  for index, linha in df.iterrows():
    Arrdist = []
    for ind2, r in df.iterrows():
        x = (linha-r)
        dist = calc(expr,x)
        Arrdist.append(dist)
    Mdist.append(Arrdist)
  return Mdist


#calcula a fitness de todos os individuos de uma pop
def Fitness(true_labels,df,pop):
    exprs = criar_expressoes(pop)
    metrics = []

    for i in range(len(pop)):
        Mdist = calc_Matriz(exprs[i],df)
        clustering = AgglomerativeClustering( metric='precomputed', linkage='average')
        labels = clustering.fit_predict(Mdist)
        Vmetric = v_measure_score(true_labels, labels)
        ind = (pop[i],Vmetric)
        metrics.append(ind)
    return sorted(metrics,key=lambda x: x[1],reverse=True)
        
#seleciona popsize individuos por torneio
def Selecao(popsize, metrics, k=2):
    rands = []
    for j in range(k):
      rng = randrange(popsize)
      rands.append(metrics.copy()[rng])
    select = (sorted(rands,key=lambda x: x[1],reverse=True))
    return select

In [8]:
#altera um bit aleatorio do numero de entrada e retorna o numero mutado e a pos alterada 
# -- Mutacao de um ponto
def mutacao(num):
    rng = randrange(0,64)
    out = bit_switch(num, rng)
    return out,rng


#troca o binario de dois individuos a partir de uma posicao aleatoria
def crossover(pai1,pai2):
    rng = randrange(0,64)
    p1_1 = bin(pai1)[2:].zfill(64)[:rng]
    p1_2 = bin(pai1)[2:].zfill(64)[rng:]

    p2_1 = bin(pai2)[2:].zfill(64)[:rng]
    p2_2 = bin(pai2)[2:].zfill(64)[rng:]

    out1 = int(p1_1 + p2_2.zfill(64-rng),2)
    out2 = int(p2_1 + p1_2.zfill(64-rng),2)
    return out1,out2


def NextGen(pais, popsize, elitsize, pCross, pMut,k=2):
    filhos = []
    Fitpais = []
    
    #copia da elite
    for indv, fit in pais[:elitsize]:
        filhos.append((indv))
        Fitpais.append(-1)

    #adicionar limite de preenchimento
    while(len(filhos) < (popsize)):
        indvs = []
        while(len(indvs)<2): #cria dois individuos
            indv = Selecao(popsize,pais,k)[0]
            indvs.append(indv)
            Fitpais.append(-1)
            if(rd.random() <= pMut ):
                filhos.append(mutacao(indv[0])[0])

        if(rd.random() <= pCross):
                c1, c2 = crossover(indvs[0][0],indvs[1][0])
                FitMedPais = np.mean([indvs[0][1],indvs[1][1]])
                filhos.append(c1)
                Fitpais.append(FitMedPais)
                filhos.append(c2)
                Fitpais.append(FitMedPais)
        
        
    return (filhos,Fitpais) #individuos sem fit, fit dos pais de crossovers

In [9]:
def DFprep(df):#add n cols
    datan = normalize(df.iloc[:,:8])
    datan = pd.DataFrame(datan, columns=df.columns[:8])
    true_labels = df['Classification']-1
    return datan,true_labels

def GP(ngen, popsize, elitsize, pCross, pMut, df, k):
    data, true_labels = DFprep(df)

    pop = Pop_start(popsize)#indv sem fit
    FitAtual = Fitness(true_labels, data, pop)# individuos com fit

    SortFit = pd.Series(sorted(FitAtual,key=lambda x: x[1],reverse=True))
    bestFit = SortFit.head(1).iloc[0][1]
    worstFit = SortFit.tail(1).iloc[0][1]
    meanFit = (SortFit.apply(lambda t: t[1]).mean())
    Nrepeats = (SortFit.value_counts() - 1).clip(lower=0).sum()
    orgulho = 0
    decepcao = 0
    results =( [{'gen'     : 0,
                'bestFit' : bestFit,
                'worstFit': worstFit,
                'meanFit' : meanFit,
                'Nrepeats': Nrepeats,
                'orgulho' : orgulho,
                'decepcao' : decepcao
                }])
    resultsDF = pd.DataFrame(results)        

    for gen in range(0,ngen):
        #cria nova geracao e calcula sua fit
        pop, FitPais = (NextGen(FitAtual,popsize,elitsize,pCross,pMut)) #individuos sem fit, fit dos pais de cada individuo fruto de crossover,-1 caso fruto de mutaçao ou elitismo 
        FitAntiga = FitAtual.copy()
        FitAtual = Fitness(true_labels, data, pop)# individuos com fit

        #contabilizando filhos melhores e piores que os pais
        decepcao = 0
        orgulho = 0
        for j in range(popsize):
            Pais = FitPais[j]
            if(Pais != -1):
                if(Pais > FitAtual[j][1]):
                    decepcao += 1
                else:
                    orgulho += 1

        #organizando informaçoes desejadas e armazena em um DF
        SortFit = pd.Series(sorted(FitAtual,key=lambda x: x[1],reverse=True))
        bestFit = SortFit.head(1).iloc[0][1]
        worstFit = SortFit.tail(1).iloc[0][1]
        meanFit = (SortFit.apply(lambda t: t[1]).mean())
        Nrepeats = repeticoes = (SortFit.value_counts() - 1).clip(lower=0).sum()
        results =( [{'gen' : gen+1,
                'bestFit' : bestFit,
                'worstFit': worstFit,
                'meanFit' : meanFit,
                'Nrepeats': Nrepeats,
                'orgulho' : orgulho,
                'decepcao' : decepcao
                }])
        resultsDF = pd.concat([resultsDF,pd.DataFrame(results)])          
    print("uma feita\n")
    return resultsDF



In [62]:
data = pd.read_csv('data/breast_cancer_coimbra_train.csv')



In [ ]:
                teste = 'r_pop30'
r = []
ngen= 30
popsize=30
elitsize=10
pCross=0.9
pMut=0.05
df=data
k=2

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.head()

uma feita



,gen,bestFit,worstFit,meanFit,Nrepeats,orgulho,decpcao
0,0,0.041181,0.000041,0.011704,0,0,0
0,1,0.083669,0.000397,0.024115,0,6,4
0,2,0.083669,0.000003,0.029897,1,4,10
0,3,0.083669,0.000003,0.043860,5,4,16
0,4,0.083669,0.000483,0.054645,11,3,21


In [106]:
#testes
nums = dividir_binario(2216832510153961809)
print(nums)
reset()
arvore = ConstructTree(nums)
folhas = armazena_folhas(arvore)
#ImprimeArvore(arvore)
' '.join(folhas)

[30, 195, 197, 108, 54, 67, 177, 81]


'x5 + ( div ( x6 , x1 ) * div ( x6 , x4 ) )'

In [107]:
#testes

# result_Spop = {} #ngen= , popsize=X, elitsize=5, pCross=0.9, pMut=0.3, df=data, k=2
# result_Ngen = {} #ngen= X , popsize= , elitsize=5, pCross=0.9, pMut=0.3, df=data, k=2

# ngen= 5
# popsize=5
# elitsize=10
# pCross=0.9
# pMut=0.05
# df=data
# k=2

# gp = GP(ngen, popsize, elitsize, pCross, pMut, df, k)

# result_Spop[popsize] = gp

In [15]:
teste = 'r_pop30'
r = []
ngen= 30
popsize=30
elitsize=10
pCross=0.9
pMut=0.05
df=data
k=2

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)

for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])

rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)


uma feita

uma feita

uma feita

uma feita

uma feita



In [16]:
teste = 'r_pop50'
r = []
ngen= 30
popsize=50
elitsize=10
pCross=0.9
pMut=0.05
df=data
k=2

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita



In [17]:
teste = 'r_pop100'
r = []
ngen= 30
popsize=100
elitsize=10
pCross=0.9
pMut=0.05
df=data
k=2

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita



In [ ]:
teste = 'r_gen30'
r = []
ngen= 30
popsize=50
elitsize=10
pCross=0.9
pMut=0.05
df=data
k=2

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita



In [18]:
teste = 'r_gen50'
r = []
ngen= 50
popsize=50
elitsize=10
pCross=0.9
pMut=0.05
df=data
k=2

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita



In [19]:
teste = 'r_gen100'
r = []
ngen= 100
popsize=50
elitsize=10
pCross=0.9
pMut=0.05
df=data
k=2

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita



In [ ]:
teste = 'r_probs'
r = []
ngen= 30
popsize=50
elitsize=10
pCross=0.6
pMut=0.3
df=data
k=2

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)
rDFdp = rDF.groupby('gen',as_index=False).std()
rDFdp.to_csv(f'./results/{teste}_dp.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita



In [11]:
teste = 'r_T5'
r = []
ngen= 30
popsize=50
elitsize=10
pCross=0.6
pMut=0.3
df=data
k=5

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)
rDFdp = rDF.groupby('gen',as_index=False).std()
rDFdp.to_csv(f'./results/{teste}_dp.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita



In [12]:
teste = 'r_nE'
r = []
ngen= 30
popsize=50
elitsize=0
pCross=0.6
pMut=0.3
df=data
k=5

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)
rDFdp = rDF.groupby('gen',as_index=False).std()
rDFdp.to_csv(f'./results/{teste}_dp.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita



In [14]:
datat = pd.read_csv('data/breast_cancer_coimbra_test.csv')

In [17]:
teste = 'r_DataT'
r = []
ngen= 30
popsize=50
elitsize=0
pCross=0.6
pMut=0.3
df=datat
k=5

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(9):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)
rDFdp = rDF.groupby('gen',as_index=False).std()
rDFdp.to_csv(f'./results/{teste}_dp.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita

uma feita

uma feita

uma feita

uma feita

uma feita



In [18]:
rDFmean.head()

,gen,bestFit,worstFit,meanFit,Nrepeats,orgulho,decepcao
0,0,0.245950,0.000000e+00,0.057288,0.0,0.0,0.0
1,1,0.229909,9.063201e-17,0.072369,1.2,7.9,10.9
2,2,0.313355,0.000000e+00,0.080215,0.5,6.0,11.2
3,3,0.361405,9.063201e-17,0.086620,0.6,5.8,13.4
4,4,0.317337,9.063201e-17,0.093947,1.2,7.8,11.0


In [10]:
#dericavao da gramatica
def derivacao(nterm, num,dp = 0):#add nvars
    if(nterm[0] != '<'):
        return 't'
    
    if(nterm == '<op>'):
        if(num%3 == 0):
            return ( '+')
        elif(num%3 == 1):
            return ('-')
        elif(num%3 == 2):
            return ('*')
        
    if(nterm == '<protop>'):
        return ('div')
            
    if(dp == 1):
        return (f'x{num%8}')
    
    if(nterm == '<start>'):
        if(num%3 == 0):
            return ( '<exp> <op> <exp>')
        if(num%3 == 1):
            return ('<exp>')
        if(num%3 == 2):
            return ( '( <exp> <op> <exp> )')
        
    if(nterm == '<exp>'):
        if(num%4 == 0):
            return ('<exp> <op> <exp>')
        elif(num%4 == 1):
            return ('( <exp> <op> <exp> )')
        elif(num%4 == 2):
            return ( '<protop> ( <term> , <term> )')
        elif(num%4 == 3):
            return ('<term>')
            

    if(nterm == '<term>'):
        return (f'x{num%11}')
        


In [11]:
def calc12 (expr, x):
    x0 = x.iloc[0]
    x1 = x.iloc[1]
    x2 = x.iloc[2]
    x3 = x.iloc[3]
    x4 = x.iloc[4]
    x5 = x.iloc[5]
    x6 = x.iloc[6]
    x7 = x.iloc[7]
    x8 = x.iloc[8]
    x9 = x.iloc[9]
    x10 = x.iloc[10]
 
    return eval(expr)

In [12]:
def calc_Matriz(expr,df):
  Mdist = []
  for index, linha in df.iterrows():
    Arrdist = []
    for ind2, r in df.iterrows():
        x = (linha-r)
        dist = calc12(expr,x)
        Arrdist.append(dist)
    Mdist.append(Arrdist)
  return Mdist

In [13]:
def DFprep(df):#add n cols
    datan = normalize(df.iloc[:,:11])
    datan = pd.DataFrame(datan, columns=df.columns[:11])
    datan = pd.concat([datan,df.iloc[:,11]],axis=1)
    datan.head()
    true_labels = df.iloc[:,11]-1
    return datan,true_labels

In [14]:
def Fitness(true_labels,df,pop):
    exprs = criar_expressoes(pop)
    metrics = []

    for i in range(len(pop)):
        Mdist = calc_Matriz(exprs[i],df)
        clustering = AgglomerativeClustering(n_clusters=7, metric='precomputed', linkage='average')
        labels = clustering.fit_predict(Mdist)
        Vmetric = v_measure_score(true_labels, labels)
        ind = (pop[i],Vmetric)
        metrics.append(ind)
    return sorted(metrics,key=lambda x: x[1],reverse=True)

In [24]:
wine = pd.read_csv('data/wineRed-train.csv',header=None,index_col=None)
wine = wine.iloc[:100]

In [25]:
teste = 'r_wineTrain'
r = []
ngen= 30
popsize=50
elitsize=0
pCross=0.6
pMut=0.3
df=wine
k=5

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)
rDFdp = rDF.groupby('gen',as_index=False).std()
rDFdp.to_csv(f'./results/{teste}_dp.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita



In [23]:
wineTs = pd.read_csv('data/wineRed-test.csv',header=None,index_col=None)
wineTs = wineTs.iloc[:100]

In [26]:
teste = 'r_wineTest'
r = []
ngen= 30
popsize=50
elitsize=0
pCross=0.6
pMut=0.3
df=wine
k=5

rDF = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
rDF.to_csv(f'./results/{teste}_0.csv',index=True)
for j in range(4):
    rDFi = GP(ngen, popsize, elitsize, pCross, pMut, df, k)
    rDFi.to_csv(f'./results/{teste}_{j+1}.csv',index=True)
    rDF = pd.concat([rDF,rDFi])
rDFmean = rDF.groupby('gen',as_index=False).mean()
rDFmean.to_csv(f'./results/{teste}_mean.csv',index=True)
rDFdp = rDF.groupby('gen',as_index=False).std()
rDFdp.to_csv(f'./results/{teste}_dp.csv',index=True)

uma feita

uma feita

uma feita

uma feita

uma feita

